In [ ]:
import sys
sys.path.append("/home/ubuntu/mlab/days/w3d3/rl_env/lib/python3.8/site-packages")
sys.path.append("/home/ubuntu/mlab/days/w3d3/rl_env/lib/python3.9/site-packages")


In [ ]:
import torch as t
from torch import nn, optim
import rl_tests
import gym
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
from IPython.display import Video
#from video_recorder import VideoRecorder
import video_recorder
import time
import collections
import random
import numpy as np
import days.

In [37]:
DEVICES = ["cuda:4", "cuda:5", "cuda:6", "cuda:7"]
DEVICE = DEVICES[0] if t.cuda.is_available() else "cpu"

In [ ]:
class Qnetwork(nn.Module):
    def __init__(self, in_size : int, hidden_size : int, out_size : int):
        super().__init__()#i presume in_size=obs_state; out_size=num_actions
        self.model = nn.Sequential(
            nn.Linear(in_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,out_size))
    def forward(self, obs): #Tensor[..., obs_shape] -> Tensor[..., num_actions]
        return self.model(obs)

rl_tests.test_q_net(Qnetwork)

test_q_net MATCH!!!!!!!!
 SHAPE (16, 30) MEAN: -0.0177 STD: 0.09063 VALS [-0.1041 0.05202 0.00405 -0.08737 0.08234 -0.1316 -0.004065 -0.1105 0.02289 -0.1047...]


In [ ]:
def make_choice(env, eps:float,net:nn.Module,obs,device="cpu"):
    if t.rand(1)<eps:
        choice = env.action_space.sample()
        #print("random choice:", choice)
    else:
        choice=t.argmax(net(obs),dim=-1).item()
        #print("model choice:", choice)

    return choice
    

In [ ]:
def evaluate(model,env,eps=0.05,device="cpu"):
    model.eval()
    obs= env.reset()
    tot_reward=0
    done=False
    with t.no_grad():
        while not done:
            obs,reward,done,_=env.step(make_choice(env,eps,model.to(device),t.tensor(obs,dtype=t.float).to(device),device))
            tot_reward+=reward
    return tot_reward
    

In [ ]:
env_name = "CartPole-v1"
env = gym.make(env_name)

In [ ]:
model = Qnetwork(env.observation_space.shape[0],64, env.action_space.n)

In [ ]:
evaluate(model,env)


9.0

In [ ]:

def run_training_loop(env, model,eps=(lambda t: 0.05),gamma=0.98,train_freq=16,batch_size=128,max_buffer_size=10000,adam_lr=1e-3,n_steps=20000,device="cpu"):
    buffer=collections.deque([],max_buffer_size)
    optimizer=optim.Adam(model.parameters(),adam_lr)
    lossfn = nn.MSELoss()
    done=True
    for i in range(batch_size):
        if done == True:
            obs=env.reset()
        action= env.action_space.sample()
        next_obs,reward,done,_=env.step(action)
        buffer.append((obs,action,reward,done,next_obs))
        obs=next_obs
    done=True
    returns = []
    current_return = None
    for i in range(n_steps):
        if done == True:
            if current_return is not None:
                returns.append(current_return)
            current_return = 0
            ob=env.reset()
        with t.no_grad():
            action =make_choice(env,eps(i/n_steps),model.to(device),t.tensor(ob,dtype=t.float).to(device),device)
            next_ob,reward,done,_=env.step(action)
            current_return += reward
            buffer.append((ob,action,reward,done,next_ob))
            ob=next_ob
        if i%train_freq==train_freq-1:
            to_train = random.sample(list(buffer),batch_size)
            obs, actions, rewards, dones, next_obs= zip(*to_train)
            obs = t.tensor(np.array(obs), dtype=t.float).to(device)
            actions = t.tensor(actions).to(device)
            rewards = t.tensor(rewards).to(device)
            dones = t.tensor(dones, dtype=t.float).to(device)
            next_obs = t.tensor(np.array(next_obs), dtype=t.float).to(device)
            with t.no_grad():
                target=rewards + (1-dones)*t.max(model(next_obs),axis = -1).values
            inpt = t.gather(model(obs),dim=-1,index=actions.unsqueeze(1)).squeeze(1)
            loss = lossfn(inpt,target)
            #if i%1000==999:
                #print(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        if i%1000==0:
            #print('current reward', len(buffer)/(sum(item[3] for item in buffer)))
            print('current return:', sum(returns[-10:])/max(1,len(returns[-10:])))
        
            
                                                                
    

In [ ]:
run_training_loop(env,model,n_steps=100000)

In [ ]:
t.tensor([evaluate(model,env,eps=0) for i in range(0,100)])

tensor([420., 500., 500., 466., 489., 500., 447., 263., 500., 500., 427., 440.,
        247., 262., 302., 425., 487., 380., 420., 432., 385., 430., 500., 437.,
        378., 450., 500., 500., 446., 500., 432., 500., 495., 365., 500., 448.,
        434., 364., 420., 247., 500., 442., 490., 500., 500., 461., 467., 408.,
        500., 500., 417., 500., 500., 500., 438., 237., 334., 500., 500., 415.,
        248., 336., 487., 500., 421., 500., 278., 500., 500., 500., 365., 412.,
        361., 438., 500., 500., 498., 378., 497., 396., 500., 465., 500., 442.,
        480., 393., 267., 241., 380., 500., 500., 453., 442., 473., 422., 500.,
        500., 500., 437., 500.])

In [ ]:
def generate_video(env,model,video_name,record=True,device="cpu"):
    state = env.reset()
    done = False
    total_reward = 0

    def show_state(env, step=0, info=""):
        plt.figure(3)
        plt.clf()
        plt.imshow(env.render(mode='rgb_array'))
        plt.axis('off')
        display.clear_output(wait=True)
        display.display(plt.gcf())

    if record:
        recorder = video_recorder.VideoRecorder(
            env, f"videos/{video_name}.mp4")

    states = 0
    while not done:
        states += 1
        if record:
            recorder.capture_frame()
        else:  
            show_state(env)
        state, reward, done, _ = env.step(make_choice(env,0,model.to(device),t.tensor(state,dtype=t.float).to(device),device)) # Take a random action
        #print(state)
        total_reward += reward

    print(f"total reward: {total_reward}")
    if record:
        recorder.close()
    return Video(f"videos/{video_name}.mp4")

In [ ]:
generate_video(env,model,"model_test_tentpole")

total reward: 500.0


In [ ]:
 Video("videos/model_test_tentpole.mp4")

In [ ]:
env_name = "Acrobot-v1"
env2 = gym.make(env_name)

In [34]:
acrobot = Qnetwork(env2.observation_space.shape[0],64, env2.action_space.n)
run_training_loop(env2, acrobot, eps=(lambda t: max(1-t*9,0.1)),gamma=0.99,train_freq=4,batch_size=128,max_buffer_size=10000,adam_lr=1e-4,n_steps=50000,device="cpu")

current return: 0.0
current return: -500.0
current return: -500.0
current return: -500.0
current return: -500.0
current return: -500.0
current return: -386.1
current return: -88.2
current return: -89.8
current return: -97.3
current return: -99.9
current return: -119.3
current return: -124.2
current return: -94.2
current return: -107.9
current return: -115.6
current return: -116.7
current return: -100.0
current return: -113.4
current return: -103.7
current return: -102.1
current return: -115.7
current return: -128.6
current return: -122.0
current return: -98.7
current return: -105.9
current return: -110.1
current return: -115.7
current return: -114.2
current return: -135.4
current return: -114.6
current return: -107.9
current return: -109.4
current return: -115.1
current return: -102.9
current return: -108.9
current return: -122.0
current return: -118.9
current return: -117.6
current return: -118.5
current return: -120.4
current return: -112.9
current return: -115.5
current return: -120

In [35]:
generate_video(env2,acrobot,"model_test_acrobot")
Video("videos/model_test_acrobot.mp4")

total reward: -110.0


In [36]:
class QnetworkBreakout(nn.Module):
    def __init__(self, obs_n_channels : int, n_action_space : int):
        super().__init__()#i presume in_size=obs_state; out_size=num_actions
        self.model = nn.Sequential(
            nn.Conv2d(obs_n_channels,32,8,stride=4),
            nn.ReLU(),
            nn.Conv2d(32,64,4,stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, n_action_space),
          )

    def forward(self, obs): #Tensor[..., obs_shape] -> Tensor[..., num_actions]
        return self.model(obs)

#rl_tests.test_q_net(Qnetwork)

test_q_net MATCH!!!!!!!!
 SHAPE (16, 18) MEAN: -0.005567 STD: 0.1015 VALS [-0.1383 -0.04243 0.05936 -0.2105 0.05156 0.1246 -0.06088 -0.1034 0.09819 0.01536...]


In [38]:
env_name = "BreakoutNoFrameskip-v0"
env3 = gym.make(env_name)
env3 = 
breakout_model = QnetworkBreakout(env3.observation_space.shape[0], env3.action_space.n)
run_training_loop(env3, breakout_model, eps=(lambda t: 1-t*(1-0.01)),gamma=0.99,train_freq=4,batch_size=32,max_buffer_size=int(1e5),adam_lr=3e-5,n_steps=int(1e6),device=DEVICE)

current return: 0.0


RuntimeError: Calculated padded input size per channel: (160 x 3). Kernel size: (8 x 8). Kernel size can't be greater than actual input size